In [1]:
from Bio import Entrez
from Bio import SeqIO
from Bio import GenBank
import re
import collections
Entrez.email = "aine.otoole@ed.ac.uk"

In [135]:
id_dict = {"M87661":"GI-NV|Hu|Norwalk|USA|1968",
           "L07418":"GI-SHV|Hu|Southampton|UK|1991",
           "U07611":"GII-HV|Hu|Hawaii|USA|1971",
           "X86557":"GII-LDV|Hu|Lordsdale|UK|1993",
           "AB074893":"GII-Sw918|Po||Japan|1997",
           "AJ011099":"GII-Jena|Bo|Jena|Germany|1980",
           "AF097917":"GIII-Newbury2|Bo|Newbury|UK|1976",
           "AF195847":"GIV-Alphatron|Hu|Alphatron|NL|1998",
           "AF414426":"GIV-FtLauderdale|Hu|Fort_Lauderdale|USA|1998",
           "AY228235":"GV-MNV1|Mu||USA|2002",
           "AJ844470":"GV1-Chiba|Hu|Chiba|JP|2004",
           "FJ875027":"GVI-Bari|Ca|Bari|IT|2007",
           "GQ443611":"GVI-Viseu|Ca|Viseu|PT|2007"    
}

In [150]:
handle = Entrez.efetch(db="nucleotide", 
                       id="M87661,L07418,U07611,X86557,AB074893,AJ011099,AF097917,AF195847,AF414426,AY228235,AJ844470,FJ875027,GQ443611", 
                       rettype="gb", retmode="xml")
id_str="M87661,L07418,U07611,X86557,AB074893,AJ011099,AF097917,AF195847,AF414426,AY228235,AJ844470,FJ875027,GQ443611"
id_list = id_str.split(',')
fw=open('../norovirus/reference_set.fasta','w')
for record in Entrez.read(handle):
    #info = '|'.join(record['GBSeq_source'].replace(' ','_').split('/'))
    c=record['GBSeq_primary-accession']
    fw.write(">{}|{}\n{}\n".format( c, id_dict[c], record['GBSeq_sequence']))
fw.close()



In [172]:
handle = Entrez.efetch(db="nucleotide", 
                       id="M87661,L07418,U07611,X86557,AB074893,AJ011099,AF097917,AF195847,AF414426,AY228235,AJ844470,FJ875027,GQ443611", 
                       rettype="gb", retmode="text")
id_str="M87661,L07418,U07611,X86557,AB074893,AJ011099,AF097917,AF195847,AF414426,AY228235,AJ844470,FJ875027,GQ443611"
id_list = id_str.split(',')
fw=open('../norovirus/reference_set.gb','w')
# for record in Entrez.read(handle):
#     #info = '|'.join(record['GBSeq_source'].replace(' ','_').split('/'))
#     c=record['GBSeq_primary-accession']
#     fw.write(">{}|{}\n{}\n".format( c, id_dict[c], record['GBSeq_sequence']))
fw.close()

for record in SeqIO.parse(handle, "gb"):
    print("{}\t{}\t{}".format(record.name, len(record), len(record.features)))
    for feature in record.features:
        if feature.type=='CDS':
            if 'gene' in feature.qualifiers:
                print feature.type, 'Gene', feature.qualifiers['gene'], feature.extract(record.seq)[:20]
            elif 'note' in feature.qualifiers:
                print feature.type, 'Note', feature.qualifiers['note'], feature.extract(record.seq)[:20]
            elif 'product' in feature.qualifiers:
                print feature.type, 'Product', feature.qualifiers['product'], feature.extract(record.seq)[:20]
            
            print feature.location

CVXRNA	7654	4
CDS Note ['orf1; sequence homologies to 2C helicase, 3C protease, and 3D RNA-dependent RNA polymerase of picornavirus'] ATGATGATGGCGTCAAAAGA
[4:5374](+)
CDS Note ['orf2'] ATGATGATGGCGTCTAAGGA
[5357:6950](+)
CDS Note ['encodes small basic protein of unknown function'] ATGGCCCAAGCCATAATTGG
[6949:7588](+)
SOUCAPPRO	7708	6
CDS Product ['polyprotein'] ATGATGATGGCGTCGAAAGA
[4:5371](+)
CDS Note ['putative'] ATGATGATGGCGTCTAAGGA
[5354:6995](+)
CDS Note ['ORF; putative'] ATGGCCCAAGCCATTATAGG
[6994:7630](+)
HCU07611	7513	4
CDS Note ['orf1'] ATGAAGATGGCGTCTAACGA
[4:5104](+)
CDS Note ['orf2'] ATGAAGATGGCGTCGAATGA
[5084:6692](+)
CDS Note ['orf3'] ATGGCTGGAGCTTTTATAGC
[6691:7471](+)
X86557	7555	7
CDS Gene ['orf1'] ATGAAGATGGCGTCTAACGA
[4:5104](+)
CDS Gene ['orf2'] ATGAAGATGGCGTCGAATGA
[5084:6704](+)
CDS Gene ['orf3'] ATGGCTGGAGCTTTCTTTGC
[6703:7510](+)
AB074893	2429	3
CDS Note ['ORF1'] TTCGAGGTGGGATTCCACTC
[<0:802](+)
CDS Note ['ORF2'] ATGAAGATGGCGTCTAGCGA
[785:2429](+)
AJ011099	7338	4

In [2]:
fw = open('references_by_orf.fasta','w')
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3
id_sp_dict = {}
orf1 = {}
orf2= {}
orf3 = {}
c = 0
len_dict = {}
terms1 = ['ORF1','orf1',' ORF1','POL','polyprotein','nonstructural polyprotein','nonstructural protein',
          'non-structural polyprotein','nonstructrual polyprotein','nsp','ORF1 polyprotein']
terms2 = ['ORF2','orf2','VP1','vp1','viral protein 1','capsid protein VP1','capsid VP1','VP1 capsid protein',
          'major capsid protein','major capsid protein VP1','capsid protein']
terms3 = ['ORF3','orf3',' orf3','VP2','vp2','viral protein 2','capsid protein VP2','capsid VP2','minor capsid protein',
          'minor structural protein','minor capsid protein VP2','small basic protein',
          'VP2 minor structural protein','small basic protein VP2','ORF3 protein',
         'truncated small basic protein']
for record in SeqIO.parse("complete_genomes.gb", "genbank"):
    c +=1
#     if c<100:
#     print record.id, len(record.seq), len(record.features)
    for f in record.features:
        if f.type=='source':
            id_sp_dict[record.id] = f.qualifiers.get('organism',f.qualifiers.get('strain'))[0]
        if f.type == 'CDS':
            feature_info = []
            for i in [f.qualifiers.get('note'), f.qualifiers.get('gene'), f.qualifiers.get('product')]:
                if i:
                    tokens = i[0].split(';')
                    for t in tokens:
                        feature_info.append(t)
#                 print feature_info
            if intersection(terms1, feature_info):
                orfseq = f.extract(record.seq)
                orf1[record.id]= orfseq
                len_dict["ORF1|" + str(record.id)]= len(orfseq)
                fw.write('>ORF1|{}\n{}\n'.format(record.id,orfseq))
            elif intersection(terms2, feature_info):
                orfseq = f.extract(record.seq)
                orf2[record.id]= orfseq
                len_dict["ORF2|" + str(record.id)]= len(orfseq)
                fw.write('>ORF2|{}\n{}\n'.format(record.id,orfseq))
            elif intersection(terms3, feature_info):
                orfseq = f.extract(record.seq)
                orf3[record.id]= orfseq
                len_dict["ORF3|" + str(record.id)]= len(orfseq)
                fw.write('>ORF3|{}\n{}\n'.format(record.id,orfseq))
            
print len(orf1), len(orf2), len(orf3), c, len(id_sp_dict)
fw.close()


1184 1176 1172 1187 1187


In [267]:
for k in len_dict:
    print k, len_dict[k]

ORF1|KJ196295.1 5094
ORF1|AB541333.1 5100
ORF3|AB933722.1 807
ORF1|KJ196276.1 5100
ORF1|KF039734.1 5370
ORF1|AB933728.1 5097
ORF1|AB933651.1 5100
ORF2|JN400614.1 1623
ORF1|AB933770.1 5100
ORF1|AB541331.1 5100
ORF3|AB933720.1 807
ORF1|EU921388.2 5100
ORF1|KF039736.1 5370
ORF3|AB541208.1 807
ORF1|AB933653.1 5100
ORF3|LC209432.1 780
ORF2|KC464497.1 1608
ORF3|JN400609.1 807
ORF3|KC013592.1 807
ORF1|AB541311.1 5100
ORF1|KF039730.1 5370
ORF2|AB541332.1 1623
ORF2|LC209438.1 1629
ORF2|MH218640.1 1644
ORF3|LC209434.1 780
ORF1|AB541313.1 5100
ORF2|KC409285.1 1623
ORF3|KC175344.1 807
ORF1|LN854572.1 5100
ORF1|KC409239.1 5100
ORF1|AB933746.1 5100
ORF2|LC209434.1 1629
ORF3|LC209438.1 780
ORF2|AB541257.1 1623
ORF2|KC464499.1 1608
ORF3|AB435514.1 627
ORF3|KC175346.1 807
ORF1|AB933647.1 5097
ORF1|AB933744.1 5100
ORF2|LC209432.1 1629
ORF1|AB541323.1 5100
ORF2|AB541259.1 1623
ORF3|LC209458.1 780
ORF2|AB541299.1 1623
ORF1|AY587989.1 5100
ORF1|AB447459.1 5100
ORF1|AB933645.1 5100
ORF1|AB541325.1 5100
ORF3

ORF2|KC409271.1 1623
ORF1|MH218594.1 5100
ORF2|MH218572.1 1647
ORF3|LC209476.1 780
ORF1|AB039775.1 5100
ORF2|MH218574.1 1647
ORF1|KC175363.1 5100
ORF2|KC175388.1 1623
ORF1|MG745998.1 5100
ORF3|MH218641.1 777
ORF1|AB933712.1 5100
ORF3|LC209470.1 780
ORF3|AB933702.1 807
ORF1|MH218625.1 5100
ORF3|LC209450.1 780
ORF2|MH218578.1 1647
ORF2|KC175384.1 1623
ORF3|AB541288.1 807
ORF1|AB447451.1 5100
ORF2|KC409261.1 1623
ORF3|KP998539.1 780
ORF1|MH218627.1 5100
ORF2|LN854568.1 1644
ORF2|MG746043.1 1629
ORF2|MH218584.1 1647
ORF2|AB933713.1 1623
ORF2|KC409302.1 1623
ORF2|AB541241.1 1623
ORF1|AB541315.1 5100
ORF1|KC464496.1 5106
ORF3|KT970371.1 780
ORF2|MH218586.1 1647
ORF2|AB933715.1 1623
ORF1|MH218649.1 5367
ORF1|AB933766.1 5097
ORF3|KJ196290.1 732
ORF1|AB447441.1 5100
ORF2|AB933735.1 1623
ORF2|AB126320.1 1644
ORF1|AB541319.1 5100
ORF2|JX989074.1 1623
ORF1|KX079488.1 5100
ORF3|KC409239.1 807
ORF1|JX486102.1 5103
ORF1|LC209468.1 3790
ORF3|KC409306.1 807
ORF3|MG745990.1 780
ORF1|AB541246.1 5100
ORF1

ORF2|KC409256.1 1623
ORF1|KP784691.1 5100
ORF2|MH218637.1 1623
ORF1|AB541243.1 5100
ORF3|AB541202.1 807
ORF3|MH218604.1 765
ORF2|AB541254.1 1623
ORF3|MF405169.1 780
ORF2|KC409258.1 1623
ORF1|KP784693.1 5100
ORF1|KC175402.1 5100
ORF1|LC209459.1 5092
ORF3|KC409279.1 807
ORF2|MH218635.1 1623
ORF1|MH218715.1 5100
ORF1|JN400619.2 5100
ORF3|MH218606.1 807
ORF2|AB541360.1 1623
ORF3|KC409244.1 807
ORF3|AB187514.1 648
ORF2|AB541224.1 1623
ORF1|KC175404.1 5100
ORF2|KC409238.1 1623
ORF3|KU561249.1 780
ORF2|KJ196280.1 1623
ORF2|EU004660.1 1626
ORF2|AY587983.1 1623
ORF2|KX158280.1 1623
ORF1|MG572731.1 4983
ORF2|MG746035.1 1629
ORF1|KC175408.1 5100
ORF3|AB541285.1 807
ORF1|AB933721.1 5097
ORF3|MH218614.1 807
ORF3|AB933653.1 807
ORF2|AY587985.1 1623
ORF2|KX158282.1 1644
ORF2|AB933694.1 1623
ORF3|KM258128.1 807
ORF3|AB541287.1 807
ORF2|MG746042.1 1629
ORF3|MH218670.1 807
ORF1|AB933723.1 5097
ORF1|JX459901.1 5100
ORF3|MH218612.1 807
ORF3|AB933651.1 807
ORF2|AY587987.1 1623
ORF1|AB541247.1 5100
ORF1|LC2

ORF1|HF952123.1 5100
ORF2|LC209479.1 1629
ORF2|LC209461.1 1629
ORF1|AB541309.1 5100
ORF1|KC894943.1 5100
ORF3|KJ196284.1 759
ORF2|EU004665.1 1626
ORF2|AB933703.1 1623
ORF2|JN400618.2 1593
ORF2|MH218720.1 1641
ORF1|AB541220.1 5100
ORF2|LC209481.1 1629
ORF1|LC209474.1 5089
ORF3|AB933670.1 807
ORF1|HF952131.1 5100
ORF2|LC209473.1 1629
ORF2|JN400605.1 1623
ORF2|JQ798158.1 1623
ORF1|EU004674.1 5064
ORF2|KC175363.1 1623
ORF2|MG049692.1 1623
ORF2|AB541320.1 1623
ORF1|KC175346.1 5100
ORF3|MH218696.1 765
ORF3|AB541265.1 807
ORF2|LC209471.1 1629
ORF3|AB447457.1 807
ORF3|KC782764.1 627
ORF2|JN400625.1 1623
ORF1|HF952135.1 5100
ORF3|AB541324.1 807
ORF3|MH218694.1 807
ORF2|KC894731.1 1671
ORF2|JN400609.1 1623
ORF2|AB933666.1 1623
ORF1|MG746014.1 5100
ORF1|AB541349.1 5100
ORF2|AB933711.1 1623
ORF3|KC464496.1 780
ORF3|MH218692.1 774
ORF3|AB541269.1 807
ORF1|MG746018.1 5100
ORF2|AB933769.1 1623
ORF3|AB447433.1 798
ORF3|AY134748.1 780
ORF2|KT970371.1 1623
ORF1|JX439819.1 5100
ORF1|MG746038.1 5100
ORF3|

ORF2|KU561254.1 1623
ORF3|AY741811.1 807
ORF3|MG745999.1 780
ORF1|EU004677.1 5064
ORF2|MG746007.1 1629
ORF1|FJ537134.1 5100
ORF1|AB447436.1 5100
ORF1|MH218699.1 5100
ORF1|KT202794.1 5100
ORF3|AB541246.1 807
ORF1|MH218615.1 5100
ORF3|KC409290.1 807
ORF2|KC409315.1 1623
ORF2|EU004677.1 1626
ORF2|AB447445.1 1623
ORF1|AB447430.1 5100
ORF2|KC175360.1 1623
ORF1|KT202796.1 5100
ORF3|JX023286.1 807
ORF2|AB541207.1 1623
ORF3|KC175380.1 807
ORF2|MH218684.1 1623
ORF2|AB447447.1 1623
ORF1|AB447432.1 5100
ORF1|AB933665.1 5097
ORF2|JQ613569.1 1608
ORF1|KC175353.1 5100
ORF3|AB541230.1 807
ORF3|GU991354.1 807
ORF2|MH218682.1 1647
ORF2|AB447441.1 1623
ORF3|KJ685415.1 807
ORF1|AB933667.1 5100
ORF2|JQ613567.1 1671
ORF2|MH218654.1 1647
ORF2|AB447461.1 1623
ORF2|KJ685414.1 1623
ORF1|KF429774.1 5370
ORF1|MH218687.1 5094
ORF1|KY442320.1 5106
ORF2|MH218602.1 1647
ORF3|JF320650.1 627
ORF3|AB447458.1 807
ORF2|MH218656.1 1641
ORF2|NC_001959.2 1593
ORF1|GQ845367.2 5100
ORF3|KC175349.1 807
ORF2|AB447453.1 1623
ORF

ORF1|AB541346.1 5100
ORF3|JQ613569.1 780
ORF2|JF320645.1 1626
ORF2|MH218613.1 1623
ORF1|MH218602.1 5100
ORF3|NC_029647.1 729
ORF1|AB541362.1 5100
ORF3|KC175406.1 807
ORF1|KC409296.1 5100
ORF3|KT202793.1 807
ORF2|KC409246.1 1623
ORF1|MH218658.1 5094
ORF2|KC175393.1 1623
ORF1|MH218581.1 5100
ORF1|MH218600.1 5100
ORF2|KC409298.1 1623
ORF2|KC409250.1 1623
ORF1|KC409290.1 5100
ORF2|KC409244.1 1623
ORF2|GU991355.1 1647
ORF1|MH218583.1 5100
ORF2|DQ369797.1 1623
ORF2|AB541311.1 1623
ORF2|EU004654.1 1626
ORF3|KC409262.1 807
ORF1|JN400624.1 5100
ORF3|KX158285.1 807
ORF1|KC409262.1 5100
ORF2|AB541283.1 1623
ORF3|MH218652.1 807
ORF1|AB933701.1 5097
ORF2|AB541313.1 1623
ORF3|AB220921.1 807
ORF1|MH218591.1 5097
ORF1|MH218636.1 5100
ORF1|AB541316.1 5100
ORF2|MG746024.1 1629
ORF3|KT970372.1 780
ORF2|MH218581.1 1647
ORF1|AB039778.1 5094
ORF2|KC409272.1 1623
ORF2|AB933764.1 1623
ORF1|MH218597.1 5100
ORF1|MH218634.1 5100
ORF1|JN400608.1 5100
ORF3|KT970374.1 780
ORF2|KC894943.1 1623
ORF2|MH218597.1 1647
O

In [3]:
hit_dict = {'ORF1':collections.defaultdict(list),
           'ORF2':collections.defaultdict(list),
           'ORF3':collections.defaultdict(list),}
with open('blast_results/results_by_orf.csv','r') as f:
    for l in f:
        tokens = l.rstrip('\n').split(',')
        subject = tokens[1].split('|')
        orf = subject[0]
        hit = subject[1]
        query = tokens[0]
        organism = id_sp_dict[hit]
        
#         if len_dict[tokens[1]] in range(int(tokens[3])-100, int(tokens[3])+100) and float(tokens[2])>95:
        hit_dict[orf][query].append((hit,organism, float(tokens[11])))
            

for orf in hit_dict:
    for norovirus in hit_dict[orf]:
        
        hit_list = hit_dict[orf][norovirus]
        sorted_hit_list = sorted(hit_dict[orf][norovirus], key = lambda x : -x[2])[:50]
        org = list(set(map(lambda x : x[1], sorted_hit_list)))
        just_org = list(set([(re.findall("(GV?I+V?\.?\d?)", i))[0].rstrip('.') for i in org if re.findall("(GV?I+V?\.?\d?)", i)]))
        
        hit_dict[orf][norovirus] = just_org
        print orf, norovirus, len(hit_dict[orf]), len(norovirus), just_org


ORF1 NORO_40-1_01_09_2014 181 20 ['GII.4', 'GII']
ORF1 NORO_165_21_06_2015 181 19 ['GII.4', 'GII']
ORF1 NORO_55_15_02_2014 181 18 ['GII']
ORF1 NORO_147-2_10_12_2015 181 21 ['GII']
ORF1 NORO_177_20_07_2015 181 19 ['GII']
ORF1 NORO_112_08_08_2014 181 19 ['GII']
ORF1 NORO_152_02_02_2015 181 19 ['GII.4', 'GII']
ORF1 NORO_111_05_08_2014 181 19 ['GII']
ORF1 NORO_238_17_02_2016 181 19 ['GII']
ORF1 NORO_80_24_09_2014 181 18 ['GI.2', 'GI.1', 'GI.6', 'GI']
ORF1 NORO_174-1_13_11_2015 181 21 ['GII.6', 'GII.2', 'GII', 'GII.1']
ORF1 NORO_100_30_06_2014 181 19 ['GII']
ORF1 NORO_198_10_11_2015 181 19 ['GI.2', 'GI.1', 'GI.6', 'GI']
ORF1 NORO_149_22_01_2015 181 19 ['GII.4', 'GII']
ORF1 NORO_218_24_12_2015 181 19 ['GII']
ORF1 NORO_132_01_05_2015 181 19 ['GII']
ORF1 NORO_82_21_10_2014 181 18 ['GI.2', 'GI.1', 'GI.6', 'GI']
ORF1 NORO_37-1_19_05_2014 181 20 ['GII.4', 'GII']
ORF1 NORO_162-1_03_05_2015 181 21 ['GII.4', 'GII']
ORF1 NORO_196_18_10_2015 181 19 ['GII.4', 'GII']
ORF1 NORO_134_07_02_2015 181 19 ['GI

In [348]:
fw = open('norovirus_genotyping.txt','w')
fw2 = open('ped_norovirus_GII.fasta','w')
fw1 = open('ped_norovirus_GI.fasta','w')
fw.write('Sequence\tORF1\tORF2\tORF3\n')
noro_dict = collections.defaultdict(list)
for orf in hit_dict:
#     print orf, len(hit_dict[orf])
    for noro in hit_dict[orf]:
#         print noro, hit_dict[orf][noro]
        noro_dict[noro].append((orf, hit_dict[orf][noro]))
    
for k in noro_dict:
#     print k
    s = sorted(noro_dict[k], key = lambda x : x[0])
    while len(s)<3:
        s.append(('',[]))
    o1 = ','.join(s[0][1])
    o2 = ','.join(s[1][1])
    o3 = ','.join(s[2][1])
    fw.write("{}\t{}\t{}\t{}\n".format(k, o1, o2, o3))
    if o1.startswith('GII'):
        fw2.write('>{}\n{}\n'.format(k,seq_dict[k]))
    else:
        fw1.write('>{}\n{}\n'.format(k,seq_dict[k]))
fw.close()
fw1.close()
fw2.close()

In [347]:
seq_dict = {}
for record in SeqIO.parse('ped_norovirus_all_seqs.fasta',"fasta"):
    seq_dict[record.id]=record.seq

    
print len(seq_dict)

181


In [261]:
for k in id_sp_dict:
    print k, id_sp_dict[k]

KJ196292.1 Norovirus GI/Hu/JP/2007/GI.P3_GI.3/Shimizu/KK2866
LC209466.1 Norovirus Hu/GII/JP/2012/GII.P16_GII.2/Hokkaido-18
AB541361.1 Norovirus Hu/GII-4/Toyama5/2007/JP
KU561254.1 Norovirus GII.17
AY485642.1 Human calicivirus NLV/GII/Langen1061/2002/DE
AB933754.1 Norovirus Hu/GII.4/Saga1/2009/JP
KF712499.1 Norovirus Hu/GII.4/NIHIC1.16/2012/USA
AB039780.1 Norwalk-like virus
MH218603.1 Norovirus GII
AB541333.1 Norovirus Hu/GII-4/Saga2/2008/JP
KY905331.1 Norovirus GII
KY934262.1 Norovirus GI
KU561252.1 Norovirus GII.17
AB541294.1 Norovirus Hu/GII-4/Miyazaki4/2007/JP
JN400603.1 Norovirus Hu/GII-4/CGMH05/2006/TW
AB933756.1 Norovirus Hu/GII.4/Saga3/2009/JP
AB447445.1 Norovirus Hu/GII-4/Toyama5/2006/JP
MH218601.1 Norovirus GII
MG745985.1 Norovirus GII
KF429789.1 Norovirus Hu/GI.1/8MC/1978/USA
KU561250.1 Norovirus GII.17
AB541300.1 Norovirus Hu/GII-4/Miyazaki9/2008/JP
KF712497.1 Norovirus Hu/GII.4/NIHIC27.2/2012/USA
AB933708.1 Norovirus Hu/GII.4/Hiroshimacity3/2011/JP
HQ009513.1 Norovirus Hu/G

AB541216.1 Norovirus Hu/GII-4/Akita4/2008/JP
EU004678.1 Murine norovirus GV/CR10/2005/USA
MG746016.1 Norovirus GII
KM258130.1 Norovirus 12-AY-1/2012/GII.4/NewOrleans/2009
AB039781.1 Norwalk-like virus
AB541218.1 Norovirus Hu/GII-4/Aomori1/2007/JP
AB541330.1 Norovirus Hu/GII-4/Saga1/2007/JP
KU757052.1 Norovirus Hu/GII/CN/2014/GII.17/FX141226-31
AB541248.1 Norovirus Hu/GII-4/Fukui4/2008/JP
AB933681.1 Norovirus Hu/GII.4/Fukui4/2010/JP
MH218690.1 Norovirus GII
JN400607.1 Norovirus Hu/GII-4/CGMH09/2006/TW
EF684915.2 Norovirus Hu/GII.4/Shellharbour/NSW696T/2006/AUS
MH218662.1 Norovirus GII
EU004679.1 Murine norovirus GV/CR11/2005/USA
KF039737.1 Norovirus Hu/GI.1/CHA6A003_20091104/2009/USA
JX048594.1 Murine norovirus
AB081723.1 Norwalk-like virus
KC409304.1 Norovirus Hu/GII/30017/2009/VNM
KU561248.1 Norovirus GII.17
KC175403.1 Norovirus Hu/Norwalk/20118/2009/VNM
KX158279.1 Norovirus GII
AB541236.1 Norovirus Hu/GII-4/Ehime1/2007/JP
MH218692.1 Norovirus GII
AB541314.1 Norovirus Hu/GII-4/Niigata

In [70]:
# c=0
fw = open("norovirus_records_1000+.fasta","w")
for record in SeqIO.parse("norovirus_records_1000+.gb", "genbank"):
    fw.write(">{}\n{}\n".format(record.id, record.seq))
# print c
fw.close()

In [285]:
print record.annotations.keys()
record.annotations["sequence_version"]

['source', 'taxonomy', 'keywords', 'references', 'accessions', 'molecule_type', 'data_file_division', 'date', 'organism', 'sequence_version', 'topology']


1

In [270]:
c = 0
organism_dict = {}
for record in SeqIO.parse("complete_genomes.gb", "genbank"):
    c+=1
    organism_dict[record.id] = record.annotations["organism"]
    

In [200]:
hit_dict = collections.defaultdict(list)
genogroup_dict = collections.defaultdict(list)
with open('blast_results/ped_norovirus_all_blast.csv','r') as f:
    for l in f:
        l = l.rstrip('\n')
        tokens = l.split(',')
        if float(tokens[3])>10 and float(tokens[10]) <0.0001:
            hit_dict[tokens[0]].append((tokens[1], tokens[10], tokens[3], tokens[4],tokens[8],tokens[9]))
sorted_hit_dict = {}
c=0
for k in hit_dict:
    c+=1
    sorted_hits =sorted(hit_dict[k], key= lambda x : float(x[1]))
    
#     if float(sorted_hits[0][1])==0 and float(sorted_hits[0][2])>6000:
#         sorted_hit_dict[k]=sorted_hits[0]
#     else:
    sorted_hit_dict[k]=sorted_hits
    print sorted_hit_dict[k][:10]
    if c<5:
        top_hits = list(set(map(lambda x : organism_dict[x[0]], sorted_hit_dict[k])))
#         print top_hits
        my_re = list(set([sorted(list(set(re.findall("(GV?I+V?\.?\d?)", i))), key = lambda x: -len(x))[0] for i in top_hits if re.findall("(GV?I+V?\.?\d?)", i)]))

        genotype= sorted(my_re, key= lambda x : -len(x))
        print genotype
        genogroup_dict[k]=genotype
            

[('MH218700.1', '0.0', '7571', '0', '1', '7571'), ('MH218705.1', '0.0', '7563', '8', '1', '7563'), ('MH218699.1', '0.0', '7560', '7', '1', '7560'), ('MH218708.1', '0.0', '7568', '14', '1', '7568'), ('MH218694.1', '0.0', '7547', '9', '1', '7547'), ('MH218711.1', '0.0', '7551', '11', '1', '7551'), ('MH218709.1', '0.0', '7517', '7', '1', '7517'), ('KC175323.1', '0.0', '7560', '52', '1', '7559'), ('KJ196279.1', '0.0', '7560', '58', '1', '7559'), ('KJ196280.1', '0.0', '7560', '58', '1', '7559')]
['GII.4', 'GII']
[('MH218634.1', '0.0', '7560', '0', '1', '7560'), ('MH218635.1', '0.0', '7559', '2', '1', '7559'), ('MH218637.1', '0.0', '7559', '3', '1', '7559'), ('MH218614.1', '0.0', '7560', '75', '1', '7560'), ('KF509946.3', '0.0', '7531', '128', '1', '7531'), ('KJ196296.1', '0.0', '7560', '142', '1', '7560'), ('KM258128.1', '0.0', '7560', '142', '1', '7560'), ('KJ196283.1', '0.0', '7560', '143', '1', '7560'), ('KM258129.1', '0.0', '7560', '143', '1', '7560'), ('KC175323.1', '0.0', '7560', '142

In [182]:
print len(genogroup_dict)
for k in genogroup_dict:
    print k, genogroup_dict[k]

153
NORO_40-1_01_09_2014 ['GII.4', 'GII']
NORO_165_21_06_2015 ['GII.4', 'GII']
NORO_55_15_02_2014 ['GII.4', 'GII.2', 'GII.3', 'GII.1', 'GII']
NORO_147-2_10_12_2015 ['GII.4', 'GII.2', 'GII.3', 'GII.1', 'GII']
NORO_112_08_08_2014 ['GII.4', 'GII.2', 'GII.3', 'GII.1', 'GII']
NORO_152_02_02_2015 ['GII.4', 'GII']
NORO_111_05_08_2014 ['GII.4', 'GII.2', 'GII.3', 'GII.1', 'GII']
NORO_238_17_02_2016 ['GII.4', 'GII.2', 'GII.3', 'GII.1', 'GII']
NORO_80_24_09_2014 ['GII.5', 'GII.1', 'GII.4', 'GI.4', 'GIII', 'GI.8', 'GI.9', 'GI.2', 'GI.3', 'GI.1', 'GI.6', 'GI.7', 'GI.5', 'GII', 'GI']
NORO_174-1_13_11_2015 ['GII.6', 'GII.7', 'GII.4', 'GII.5', 'GII.2', 'GII.1', 'GII']
NORO_100_30_06_2014 ['GII.4', 'GII.2', 'GII.3', 'GII.1', 'GII']
NORO_198_10_11_2015 ['GII.3', 'GII.4', 'GII.1', 'GI.5', 'GI.4', 'GIII', 'GI.8', 'GI.9', 'GI.2', 'GI.3', 'GI.1', 'GI.6', 'GI.7', 'GII', 'GI']
NORO_149_22_01_2015 ['GII.4', 'GII']
NORO_218_24_12_2015 ['GII.4', 'GII.2', 'GII.3', 'GII.1', 'GII']
NORO_132_01_05_2015 ['GII.4', 'GI

In [4]:
from ete3 import Tree

In [7]:
t = Tree('all_norovirus_seqs.aligned.tree.nw',format=0)
print t


                  /-NORO_170_23_09_2014
                 |
                 |         /-NORO_171_09_03_2015
               /-|      /-|
              |  |   /-|   \-NORO_174_03_06_2015
              |  |  |  |
              |   \-|   \-NORO_174-1_13_11_2015
            /-|     |
           |  |     |   /-NORO_68-7_18_03_2015
           |  |      \-|
         /-|  |         \-NORO_182_19_09_2015
        |  |  |
        |  |   \-NORO_172_02_04_2015
        |  |
      /-|   \-NORO_229_12_01_2016
     |  |
     |  |   /-NORO_173_08_05_2015
     |  |  |
     |  |  |         /-NORO_176-1_17_12_2015
     |   \-|      /-|
   /-|     |   /-|   \-NORO_208_11_12_2015
  |  |     |  |  |
  |  |      \-|   \-NORO_215_18_12_2015
  |  |        |
  |  |         \-NORO_209_15_12_2015
  |  |
  |  |   /-NORO_197_04_11_2015
  |   \-|
  |     |   /-NORO_200_25_11_2015
  |      \-|
  |         \-NORO_236_29_01_2016
  |
  |               /-NORO_179_03_08_2015
  |            /-|
  |           |  |   /-NORO_19

In [10]:
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio import AlignIO

aln = AlignIO.read(open('all_norovirus_seqs.aligned.fasta'), 'fasta')
calculator = DistanceCalculator('identity')
dm = calculator.get_distance(aln)
print dm

NORO_100_30_06_2014	0
NORO_99_09_06_2014	0.0	0
NORO_102_02_07_2014	0.00215439856373	0.00215439856373	0
NORO_106_10_07_2014	0.00287253141831	0.00287253141831	0.000718132854578	0
NORO_107_11_07_2014	0.00275284260922	0.00275284260922	0.000598444045482	0.000119688809096	0
NORO_104_07_07_2014	0.00287253141831	0.00287253141831	0.000718132854578	0.00119688809096	0.00107719928187	0
NORO_108_17_07_2014	0.00502692998205	0.00502692998205	0.0033512866547	0.00406941950928	0.00394973070018	0.00406941950928	0
NORO_109_18_07_2014	0.00430879712747	0.00430879712747	0.00263315380012	0.0033512866547	0.0032315978456	0.0033512866547	0.00143626570916	0
NORO_113_04_09_2014	0.00694195092759	0.00694195092759	0.00526630760024	0.00598444045482	0.00586475164572	0.00598444045482	0.00550568521843	0.00502692998205	0
NORO_119_27_11_2014	0.00873728306403	0.00873728306403	0.00706163973668	0.00777977259126	0.00766008378217	0.00777977259126	0.00730101735488	0.00682226211849	0.00311190903651	0
NORO_105_05_07_2014	0.0106523

In [13]:
fw = open('distance_matrix_all_noro.txt','w')
fw.write(str(dm))
fw.close()

In [12]:
for i in dm:
    print i

[0,
 0.0,
 0.002154398563734339,
 0.0028725314183123407,
 0.002752842609216044,
 0.0028725314183123407,
 0.0050269299820466795,
 0.004308797127468567,
 0.006941950927588314,
 0.00873728306403354,
 0.010652304009575064,
 0.00670257330939561,
 0.007779772591262724,
 0.00789946140035902,
 0.009455415918611654,
 0.012687013764212995,
 0.015559545182525447,
 0.012567324955116699,
 0.036026331538001166,
 0.011968880909634994,
 0.013405146618791108,
 0.01460203470975463,
 0.021184919210053832,
 0.02082585278276483,
 0.02166367444643924,
 0.02369838420107717,
 0.022860562537402762,
 0.02333931777378817,
 0.02333931777378817,
 0.024057450628366284,
 0.022860562537402762,
 0.022860562537402762,
 0.02298025134649906,
 0.023099940155595466,
 0.02417713943746258,
 0.0191502094554159,
 0.01926989826451231,
 0.01926989826451231,
 0.01926989826451231,
 0.0191502094554159,
 0.0191502094554159,
 0.019389587073608605,
 0.019389587073608605,
 0.019389587073608605,
 0.019509275882705013,
 0.019389587073608